<b>tasks:</b>
1) Create Logistic Regression Class without using SKlearn methods

2) Create Gradient Descent

3) Create RMSProp

4) Nesterov–accelerated Adaptive Moment Estimation

In [267]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

### Own class LogisticRegression:

- own basics methods: fit, predict, score

- method of optimizing is Gradient descent

In [268]:
# Class LogisticRegression 
class LogisticRegression:
    
    def __init__(self, epoches=20, learning_rate=0.0001):
        self.EPOCHS = epoches               # Количество эпох
        self.LEARNING_RATE = learning_rate  # Скорость обучения
        
        self.costs = []    # массив оценок MSE
        self.params = []   # массив параметров
        self.preds = []    # массив предсказаний
        self.list_params_gd = []
        
        self.threashold = 0.5
        
        
    #sigmoid function (функция активации от x)
    def sigm(self, X, W):
        return 1/(1 + np.exp(-np.dot(X.T,W)))
    
    
    #Optimizing by GD
    def fit(self, Y, X):
        # X + нулевой вес
        X_np = np.c_[np.array(X), np.ones(len(X))].T       # X в numpy массив [5x100]
        
        Y_np = np.asmatrix(Y).T
        
        # Начальное приближение весов
        W = np.random.normal(size=(X_np.shape[0],1), loc=1, scale=0)     # Матричца весов [5x1]
        
        # создание копии, т.к. иначе будем перезапиывать по ссылке
        self.list_params_gd = [W.copy()]

        for _ in range(self.EPOCHS):
            predictions = sigm(X_np, W)
            self.preds.append(predictions)
            
            cost = self.logloss(X_np, Y_np, W)
            self.costs.append(cost)
            
            #формулу градиента: взял отсюда https://habr.com/ru/articles/472300/
            grad = X_np*(Y_np - self.sigm(X_np, W))
            W -= -self.LEARNING_RATE*grad

            self.list_params_gd.append(W.copy())
            
    
    def predict(self, X):
        X_np = np.c_[np.array(X), np.ones(len(X))].T
        W = self.list_params_gd[-1]
        return np.round(sigm(X_np, W))
    
    
    #accuracy
    def score(self, X, Y):  
        predicts_res = self.predict(X)
        tp_tn = 0
        fp_fn = 0
        for i in range(len(Y)):
            if np.array(Y)[i] == predicts_res[i].item():
                tp_tn += 1
            else:
                fp_fn += 1
        return tp_tn / (tp_tn + fp_fn)
    
    
    def logloss(self, X,Y,W):
        cost = 0
        sigm_matr = sigm(X, W)
        for i in range(len(Y)):
            cost += (Y[i].item() * sigm_matr[i].item() + (1-Y[i].item()) * np.log(1-sigm_matr[i].item()))
        return -cost 
    
    #Следующие два метода веротнее всего реализовал неверно (изменения от fit отмечены с комментарием ->->)
    
    # RMSProp
    def fit_RMSProp(self, Y, X, y_nesterov=0.9):
        # X + нулевой вес
        X_np = np.c_[np.array(X), np.ones(len(X))].T       # X в numpy массив [5x100]
        
        Y_np = np.asmatrix(Y).T
        
        # Начальное приближение весов
        W = np.random.normal(size=(X_np.shape[0],1), loc=1, scale=0)     # Матричца весов [5x1]
        
        # создание копии, т.к. иначе будем перезапиывать по ссылке
        self.list_params_gd = [W.copy()]
        
        # ->-> начальные значения накопленного среднего
        cumulative_middle_grad = np.zeros(shape=(X_np.shape[0],1))
        # ->-> сглаживающий параметр
        e = 0.0001
        
        for _ in range(self.EPOCHS):
            predictions = sigm(X_np, W)
            self.preds.append(predictions)
            
            cost = self.logloss(X_np, Y_np, W)
            self.costs.append(cost)
            
            #формулу градиента: взял отсюда https://habr.com/ru/articles/472300/ 
            grad = X_np*(Y_np - self.sigm(X_np, W))
            
            # ->->
            cumulative_middle_grad = y_nesterov*cumulative_middle_grad + (1-y_nesterov)*np.power(self.LEARNING_RATE*grad, 2)
            # ->->
            W -= self.LEARNING_RATE*grad/np.sqrt(cumulative_middle_grad+e)

            self.list_params_gd.append(W.copy())
        
    
    # Nesterov Accelerated Gradient
    def fit_NAG(self, Y, X, y_nesterov=0.9):
        # X + нулевой вес
        X_np = np.c_[np.array(X), np.ones(len(X))].T       # X в numpy массив [5x100]
        
        Y_np = np.asmatrix(Y).T
        
        # Начальное приближение весов
        W = np.random.normal(size=(X_np.shape[0],1), loc=1, scale=0)     # Матричца весов [5x1]
        
        # создание копии, т.к. иначе будем перезапиывать по ссылке
        self.list_params_gd = [W.copy()]
        
        # ->-> начальные значения накопленного среднего
        cumulative_middle_grad = np.zeros(shape=(X_np.shape[0],1))
        
        for _ in range(self.EPOCHS):
            predictions = sigm(X_np, W)
            self.preds.append(predictions)
            
            cost = self.logloss(X_np, Y_np, W)
            self.costs.append(cost)
            
            #формулу градиента: взял отсюда https://habr.com/ru/articles/472300/
            grad = X_np*(Y_np - self.sigm(X_np, W))
            
            # ->->
            cumulative_middle_grad = y_nesterov*cumulative_middle_grad + self.LEARNING_RATE*grad
            # ->->
            W -= cumulative_middle_grad

            self.list_params_gd.append(W.copy())
    

## Load iris dataset. There are 2 classes: Versicolor(0) and Virginica(1)

In [277]:
from sklearn.datasets import load_iris

iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['species'] = iris.target
df = df[(df['species'] == 1) | (df['species'] == 2)]
df['species'] = df['species'].replace(to_replace= [1, 2], value = [0, 1])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
50,7.0,3.2,4.7,1.4,0
51,6.4,3.2,4.5,1.5,0
52,6.9,3.1,4.9,1.5,0
53,5.5,2.3,4.0,1.3,0
54,6.5,2.8,4.6,1.5,0


### Creating test and data sets. Using own LogisticRegression

In [274]:
X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
Y = df['species']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

logreg = LogisticRegression(epoches=20, learning_rate=0.001)
logreg.fit(y_train, X_train)

predict = logreg.predict(X_test)
print('accuracy ', logreg.score(X_test, y_test))


accuracy  0.75


### Nesterov Accelerated Gradient

In [275]:
X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
Y = df['species']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

logreg = LogisticRegression(epoches=20, learning_rate=0.001)
logreg.fit_NAG(y_train, X_train)

predict = logreg.predict(X_test)
print('accuracy ', logreg.score(X_test, y_test))

accuracy  0.4


### RMSProp 

In [276]:
X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
Y = df['species']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

logreg = LogisticRegression(epoches=20, learning_rate=0.001)
logreg.fit_RMSProp(y_train, X_train)

predict = logreg.predict(X_test)
print('accuracy ', logreg.score(X_test, y_test))

accuracy  0.4


<b>Реализованы следующие кейсы:</b>
- Создан класс Логистической регрессии (аналог от библиотеки SKlearn)
- Класс включает реализованные методы: fit, predict, score и прочие вспомогательные.
- Класс fit реализует метод отпимизации на основе градиентного спуска.
- Дополнительно реализованы методы с продвинутые алгоритмы градиентного спуска: Nesterov Accelerated Gradient и RMSProp

<b>Вывод:</b>
- не уверен в правильности реализации всех пунктов, работа алгоритма очень сильно может зависить от начальных параметров. Наибольшая accuracy показал алгоритм стандартного градиентного спуска. Время работы у всех трех методов примерно равна.


